In [ ]:
# Import packages as needed. Maybe PIL?
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
filename = "/datatable.csv"

datalist = np.loadtxt(filename)

for data, label in datalist:
    pylab.plot( data[:,0], data[:,1], label=label )

pylab.legend()
pylab.title("Title of Plot")
pylab.xlabel("X Axis Label")
pylab.ylabel("Y Axis Label")